## Image Export for Ilastik Training

Export a small number of random images from each experiment (as max-z BF projections) for annotation and classifier training.

In [5]:
%matplotlib inline
%load_ext dotenv
%dotenv env.sh
%run source/utils.py
import matplotlib
import matplotlib.pyplot as plt
from skimage import exposure
import pandas as pd
import numpy as np
import cytokit
from cytokit.image.ops import describe
import os
import os.path as osp
import tqdm

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
df = pd.read_csv('experiments.csv')
df.head()

,name,conf,dir,grid,n_ch,n_tiles,n_z,cond,z_pitch
0,sta-00.125-20um-s-XY01,experiment.yaml,0125uMsta-20um-grids,XY01,3,27,8,sta-00.125-20um-s,200
1,sta-00.125-20um-s-XY02,experiment.yaml,0125uMsta-20um-grids,XY02,3,27,7,sta-00.125-20um-s,200
2,sta-00.125-20um-s-XY03,experiment.yaml,0125uMsta-20um-grids,XY03,3,27,9,sta-00.125-20um-s,200
3,sta-00.250-20um-s-XY01,experiment.yaml,025uMsta-20um-grids,XY01,3,27,8,sta-00.250-20um-s,200
4,sta-00.250-20um-s-XY02,experiment.yaml,025uMsta-20um-grids,XY02,3,27,7,sta-00.250-20um-s,200


In [3]:
export_dir = '/lab/data/spheroid/ilastik/mc38/training/20x-s25-spheroid-body-05-sta-titration/bf-maxz'
export_dir

'/lab/data/spheroid/ilastik/mc38/training/20x-s25-spheroid-body-05-sta-titration/bf-maxz'

In [6]:
np.random.seed(0)

def export_images(exp_name, images, export_dir, n_images=3):
    idx = np.random.choice(np.arange(len(images)), size=n_images, replace=False)
    images = [images[i] for i in idx]
    for img, ti in images:
        # Target size is 25% in both dimensions
        img = transform.rescale(img, .25, anti_aliasing=True, multichannel=False, mode='constant', preserve_range=True)
        path = osp.join(export_dir, '{}-T{:03d}-X{:03d}-Y{:03d}.tif'.format(exp_name, ti.tile_index+1, ti.tile_x+1, ti.tile_y+1))
        if not osp.exists(osp.dirname(path)):
            os.makedirs(osp.dirname(path))
        assert 0 <= img.min() <= img.max() <= 1
        img = exposure.rescale_intensity(img, in_range=(0, 1), out_range=np.uint8).astype(np.uint8)
        # Save image with original contrast, but downscaled to 25% (50% here and 50% in the original acquisition) with 8-bit depth
        ck_io.save_image(path, img)

for i, r in tqdm.tqdm(df.iterrows(), total=len(df)):
    config = get_config(r)
    raw_dir = osp.join(os.environ['EXP_GROUP_RAW_DIR'], r['dir'], r['grid'])
    images = list(get_channel_maxz_images(config, raw_dir, 'BF'))
    export_images(r['name'], images, export_dir)

 62%|██████▎   | 15/24 [21:59<10:56, 72.99s/it]

FileNotFoundError: [Errno 2] No such file or directory: '/lab/data/spheroid/20190703-sta-titration/raw/4uMsta-grids/XY01/1_XY01_00001_Z001_CH2.tif'